In [1]:
from officelib.xllib import *
from officelib.const import xlDown, xlToRight, xlA1
from hello.pid.ag_pid.agpid import *
import types

In [2]:
# import subprocess
# # subprocess.call("tskill.exe excel")
# xl = Excel()
# for rf in xl.RecentFiles:
#     if rf.Name.endswith("80L Test Compiled Data.xlsb"):
#         wb = rf.Open()
#         break
# else:
#     raise ValueError("Not Found")
# ws = wb.Worksheets("Sheet1")
# cells = ws.Cells

In [12]:
xl = Excel()
wb = xl.Workbooks.Add()
ws = wb.Worksheets(1)
cells = ws.Cells

In [13]:
def make_summary(p, i, d, start_mode, st, sp, vol, beta, alpha, gamma, linearity, new_name):
    paste_cell.Offset(1, 1).Value = n
    paste_cell.Offset(1, 2).Value = p
    paste_cell.Offset(1, 3).Value = i
    paste_cell.Offset(1, 4).Value = d
    paste_cell.Offset(1, 5).Value = start_mode
    paste_cell.Offset(1, 6).Value = st
    paste_cell.Offset(1, 7).Value = sp
    paste_cell.Offset(1, 8).Value = vol
    paste_cell.Offset(1, 9).Value = beta
    paste_cell.Offset(1, 10).Value = alpha
    paste_cell.Offset(1, 11).Value = gamma
    paste_cell.Offset(1, 12).Value = linearity
    paste_cell.Offset(1, 13).Value = new_name

    
def get_value(r, c, f="var"):
    v = cell.Offset(r, c).Value
    if v is None:
        raise ValueError("No value for %s" % f)
    return v

first = cells.Range("A1")
cell = first
ws2 = wb.Worksheets("Sheet2")
ws2.UsedRange.Clear()
paste_cell = ws2.Cells.Range("B3")
header = ws2.Cells.Range(paste_cell.Offset(0, 1), paste_cell.Offset(0, 9))
header.Value = [("Test #", "PGain", "Itime", "DTime", "Start Mode", 
                 "StartPV", "SP", "Volume", "Beta", "Alpha", "Gamma", 
                 "Linearity", "Name")]
n = 1
runner = PIDRunner((), "compiled_data.xlsx")
while True:
    v = cell.Value
    if not v:
        break
    name, start, setpoint = v.split()
    _, st = start.split(":")
    _, sp = setpoint.split(":")
    print("Exporting data for", name, start, setpoint)
    p = get_value(2, 2, "pgain")
    i = get_value(3, 2, "itime")
    d = get_value(4, 2, "dtime")
    # vol = get_value(4, 4, "volume")
    vol = 12
    beta = get_value(4, 4, 'beta')
    alpha = get_value(5, 2, "alpha")
    gamma = get_value(5, 4, "gamma")
    linearity = get_value(6, 2, "linearity")
    new_name = " ".join(v.split()[1:]) + " Vol:%d" % vol + " Beta:%.1f"%beta
    #new_name = "Test #%d"%n
    start_mode, end_mode = name.split("To")
    make_summary(p, i, d, start_mode, st, sp, vol, beta, alpha, gamma, linearity, new_name)
    paste_cell = paste_cell.Offset(2, 1)
    
    if name == "AutoToAuto":
        klass = TempAutoToAutoTest
    elif name == "OffToAuto":
        klass = TempOffToAutoTest
    elif name == "ManToAuto":
        klass = TempManToAutoTest
    elif name == "ManToMan":
        klass = TempManToManTest
    else:
        raise ValueError(name)
    pi = PIDInfo(p=p, i=i, d=d, alpha=alpha, gamma=gamma, linearity=linearity, beta=beta)
    t = klass(pi, st, sp, "", None, new_name)
    
    def sort(self):
        other = "%s Beta %.1f" % (self.SHORT_NAME, self.pid_info.beta)
        return self.pid_info.p, self.pid_info.i, self.pid_info.d, other
    
    t.get_chart_key = types.MethodType(sort, t)
    
    tl_data = cell.Offset(8, 1)
    br_data = tl_data.End(xlDown).End(xlToRight)
    t.data = cells.Range(tl_data, br_data).Value
    t.metadata = {'Volume': vol}
    
    runner.add_test(t)
    n += 1
    cell = cell.Offset(1, 6)
    
runner._init_xl()
runner.add_to_mru = False
with screen_lock(runner._xl):
    runner.plotall(30)
    runner.chartbypid()
runner._wb.Save()
print("Done")

Exporting data for OffToAuto Start:24.0 SP:37.0
Exporting data for AutoToAuto Start:37.0 SP:35.0
Exporting data for AutoToAuto Start:35.0 SP:37.0
02/23/17 11:57:51 INFO <plotall>: Copying test data to compiled_data.xlsx
02/23/17 11:57:51 INFO <plotall>: Copying data for test 1 of 3
02/23/17 11:57:52 INFO <plotall>: Copying data for test 2 of 3
02/23/17 11:57:52 INFO <plotall>: Copying data for test 3 of 3
02/23/17 11:57:52 INFO <plotall>: Done copying data for 3 tests.
02/23/17 11:57:52 INFO <_new_chart>: Initializing chart
02/23/17 11:57:52 INFO <_new_chart>: Initializing chart
02/23/17 11:57:53 INFO <chartbypid>: Formatting Chart: P: 70.0 I: 20.0 D:0.0 O2A Beta 0.0
02/23/17 11:57:53 INFO <chartbypid>: Formatting Chart: P: 70.0 I: 20.0 D:0.0 A2A Beta 0.0
Done
